In [1]:
# Installing necessary packages

!pip install openai-whisper spacy wikipedia
!python -m spacy download en_core_web_sm
!pip install moviepy
!pip install opencv-python-headless
!pip install --upgrade google-cloud-vision



Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/803.2 kB ? eta -:--:--
     ------------------------------------- 803.2/803.2 kB 12.0 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
   --------------------------------------

In [4]:
!apt-get update
!apt-get install -y ffmpeg


Reading package lists... Done
E: Could not open lock file /var/lib/apt/lists/lock - open (13: Permission denied)
E: Unable to lock directory /var/lib/apt/lists/
W: Problem unlinking the file /var/cache/apt/pkgcache.bin - RemoveCaches (13: Permission denied)
W: Problem unlinking the file /var/cache/apt/srcpkgcache.bin - RemoveCaches (13: Permission denied)
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


In [6]:
!pip install imageio[ffmpeg]



In [7]:
import imageio_ffmpeg
print(imageio_ffmpeg.get_ffmpeg_exe())


/opt/conda/lib/python3.10/site-packages/imageio_ffmpeg/binaries/ffmpeg-linux-x86_64-v7.0.2


In [4]:
import os
os.environ["PATH"] += os.pathsep + "/path/to/ffmpeg/bin"


In [1]:
import os
print(os.path.exists("US.mp4"))


True


In [2]:
from moviepy import VideoFileClip

video = VideoFileClip("US.mp4")
video.audio.write_audiofile("audio.wav")


MoviePy - Writing audio in audio.wav


MoviePy - Done.


In [2]:
ffmpeg -version

NameError: name 'ffmpeg' is not defined

In [3]:
import shutil
print(shutil.which("ffmpeg"))


/usr/bin/ffmpeg


In [3]:
import whisper

model = whisper.load_model("turbo")
result = model.transcribe("audio.wav")
print(result["text"])

100%|█████████████████████████████████████| 1.51G/1.51G [07:07<00:00, 3.78MiB/s]
C:\Users\tm0792\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\whisper\transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


FileNotFoundError: [WinError 2] The system cannot find the file specified

In [6]:
!pip install opencv-python-headless


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 MB 103.7 MB/s eta 0:00:0000:010:01


In [1]:
import cv2
import os

video_path = "US.mp4"
output_dir = "frames/"
os.makedirs(output_dir, exist_ok=True)

cap = cv2.VideoCapture(video_path)
frame_rate = 1  # one frame per second

i = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    if int(cap.get(1)) % (30 * frame_rate) == 0:
        filename = os.path.join(output_dir, f"frame_{i}.jpg")
        cv2.imwrite(filename, frame)
        i += 1
cap.release()


In [3]:
!pip install --upgrade google-cloud-vision


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.9/527.9 kB 16.0 MB/s eta 0:00:00


In [1]:
from google.cloud import vision
client = vision.ImageAnnotatorClient()


In [5]:
from google.cloud import vision

client = vision.ImageAnnotatorClient()
response = client.label_detection(
    image=vision.Image(source=vision.ImageSource(image_uri="gs://cloud-samples-data/vision/label/setagaya.jpeg"))
)
for label in response.label_annotations:
    print(label.description)


PermissionDenied: 403 Cloud Vision API has not been used in project 1038404886674 before or it is disabled. Enable it by visiting https://console.developers.google.com/apis/api/vision.googleapis.com/overview?project=1038404886674 then retry. If you enabled this API recently, wait a few minutes for the action to propagate to our systems and retry. [reason: "SERVICE_DISABLED"
domain: "googleapis.com"
metadata {
  key: "activationUrl"
  value: "https://console.developers.google.com/apis/api/vision.googleapis.com/overview?project=1038404886674"
}
metadata {
  key: "consumer"
  value: "projects/1038404886674"
}
metadata {
  key: "containerInfo"
  value: "1038404886674"
}
metadata {
  key: "service"
  value: "vision.googleapis.com"
}
metadata {
  key: "serviceTitle"
  value: "Cloud Vision API"
}
, locale: "en-US"
message: "Cloud Vision API has not been used in project 1038404886674 before or it is disabled. Enable it by visiting https://console.developers.google.com/apis/api/vision.googleapi

In [2]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'path/to/your/service-account-key.json'


ModuleNotFoundError: No module named 'google.colab'

In [4]:
from google.cloud import vision
client = vision.ImageAnnotatorClient()

def detect_labels(path):
    with open(path, 'rb') as img_file:
        content = img_file.read()
    image = vision.Image(content=content)
    response = client.label_detection(image=image)
    return [label.description for label in response.label_annotations]

# Run on all frames
frame_keywords = []
for img in os.listdir(output_dir):
    frame_keywords.extend(detect_labels(os.path.join(output_dir, img)))


NameError: name 'os' is not defined